# Phylogenetically informed MSA-transformers

In [ ]:
import pandas as pd
import numpy as np
import ete3
import tarfile
import re
from Bio import AlignIO


data_dir = 'data'
tree_archive = f'{data_dir}/Tree.tar.gz'
msa_archive = f'{data_dir}/MSAs.tar.gz'

In [28]:
# Get studied proteins names
# Get proteins names from the phylo tree archive
tree_archive = 'data/Tree.tar.gz'
pattern = r'Tree/(\S+)_pruned.tre'
tree_proteins = [re.search(pattern, tree).group(1) for tree in tarfile.open(tree_archive).getnames() if "/" in tree]

# Get proteins names from the MSA archive
msa_archive = "data/MSAs.tar.gz"
pattern = r"MSAs/(\S+)"
msa_proteins = [re.search(pattern, msa).group(1) for msa in tarfile.open(msa_archive).getnames() if "/" in msa]

# Get proteins names with MSA and tree
proteins = set(tree_proteins) & set(msa_proteins)

print(f"Proteins with MSA and tree: {len(proteins)}")

Proteins with MSA and tree: 16761


In [41]:
# Get the A0A183 protein MSA
with open(tree_archive, 'rb') as f:
    with tarfile.open(fileobj=f, mode='r:gz') as tar:
        tree = tar.getmember('Tree/Q92870_pruned.tre')
        tree = tar.extractfile(tree)
        tree = tree.read().decode('utf-8')

In [ ]:
import ete3

def get_species_by_distance_from_human(newick_string, species_list):
    """
    Returns species ordered by increasing evolutionary distance from Homo sapiens.
    """
    tree = ete3.Tree(newick_string)
    
    # Find the Homo sapiens node
    human_node = None
    for leaf in tree.get_leaves():
        if leaf.name == "Homo_sapiens":
            human_node = leaf
            break
    
    if not human_node:
        print("Warning: Homo sapiens not found in the tree. Returning original species list.")
        return [sp for sp in species_list if sp == "Homo_sapiens"] + [sp for sp in species_list if sp != "Homo_sapiens"]
    
    # Get all nodes in the tree
    all_leaves = tree.get_leaves()
    
    # Calculate distance from each leaf to the human node
    distances = []
    for leaf in all_leaves:
        if leaf.name in species_list:
            # Calculate the evolutionary distance
            distance = tree.get_distance(human_node, leaf)
            distances.append((leaf.name, distance))
    
    # Sort by distance (ascending)
    distances.sort(key=lambda x: x[1])
    
    # Extract just the species names in order of distance
    ordered_species = [item[0] for item in distances]
    
    return ordered_species

def reorder_msa_by_human_distance(msa_file, tree_file, output_file, format="fasta"):
    """
    Reorder sequences in an MSA based on evolutionary distance from Homo sapiens.
    
    Parameters:
    - msa_file: Path to the MSA file
    - tree_file: Path to the Newick tree file
    - output_file: Path to write the reordered MSA
    - format: MSA file format (default: fasta)
    """
    # Read the alignment
    alignment = AlignIO.read(msa_file, format)
    
    # Read the tree
    with open(tree_file, 'r') as f:
        newick_string = f.read().strip()
    
    # Extract species names from sequence IDs
    seq_to_species = {}
    species_in_msa = set()
    for record in alignment:
        # Extract species name from the sequence ID (modify as needed for your naming convention)
        species = record.id.split('|')[0]  # Adjust based on your sequence ID format
        seq_to_species[record.id] = species
        species_in_msa.add(species)
    
    # Get species ordered by distance from humans
    ordered_species = get_species_by_distance_from_human(newick_string, list(species_in_msa))
    
    # Create a dictionary to store sequences by species
    species_to_records = {}
    for record in alignment:
        species = seq_to_species[record.id]
        if species not in species_to_records:
            species_to_records[species] = []
        species_to_records[species].append(record)
    
    # Create a new alignment with sequences ordered by distance from humans
    new_records = []
    for species in ordered_species:
        if species in species_to_records:
            new_records.extend(species_to_records[species])
    
    # Write the reordered alignment
    AlignIO.write(AlignIO.MultipleSeqAlignment(new_records), output_file, format)
    
    print(f"Reordered alignment written to {output_file}")
    print("Species ordered by distance from Homo sapiens:")
    for species in ordered_species:
        print(f"- {species}")
    
    return new_records

# Example usage
if __name__ == "__main__":
    # Example for simple list ordering
    newick_string = "((((((Pan_troglodytes,Pan_paniscus),Homo_sapiens),Gorilla_gorilla_gorilla),Pongo_abelii),Nomascus_leucogenys);"
    species_in_my_msa = ["Homo_sapiens", "Pan_troglodytes", "Gorilla_gorilla_gorilla", "Pan_paniscus", "Pongo_abelii"]
    
    ordered_species = get_species_by_distance_from_human(newick_string, species_in_my_msa)
    print("Species ordered by distance from Homo sapiens:")
    for species in ordered_species:
        print(f"- {species}")
    
    # Uncomment to run the full MSA reordering
    # msa_file = "proteins_alignment.fasta"
    # tree_file = "primate_phylogeny.newick"
    # output_file = "reordered_alignment.fasta"
    # reordered_records = reorder_msa_by_human_distance(msa_file, tree_file, output_file)

In [42]:
ete3_tree = ete3.Tree(tree, format=1)

In [43]:
print(ete3_tree)


                                 /-Dipodomys_ordii
                              /-|
                             |   \-Castor_canadensis
                             |
                             |               /-Mesocricetus_auratus
                             |            /-|
                             |         /-|   \-Cricetulus_griseus
                             |        |  |
                             |      /-|   \-Microtus_ochrogaster
                           /-|     |  |
                          |  |     |  |   /-Neotoma_lepida
                          |  |     |   \-|
                          |  |     |      \-Peromyscus_maniculatus_bairdii
                          |  |   /-|
                          |  |  |  |         /-Mus_musculus
                          |  |  |  |      /-|
                          |  |  |  |   /-|   \-Mus_spicilegus
                          |   \-|  |  |  |
                        /-|     |   \-|   \-Mus_caroli
                      

In [32]:
# Get the A0A183 protein MSA
with open(msa_archive, 'rb') as f:
    with tarfile.open(fileobj=f, mode='r:gz') as tar:
        msa = tar.getmember('MSAs/A0A183')
        msa = tar.extractfile(msa)
        msa = msa.read().decode('utf-8').splitlines()

In [33]:
msa

['>sp|A0A183|LCE6A_HUMAN Late cornified envelope protein 6A OS=Homo sapiens OX=9606 GN=LCE6A PE=1 SV=1',
 'MSQQKQQSWKPPNVPKCSPPQRSNPCLAPYSTPCGAPHSEGCHSSSQRPEVQKPRRARQKLRCLSRGTTYHCKEEECEGD',
 '>sp|A0A183|LCE6A_HUMAN Late cornified envelope protein 6A OS=Homo sapiens OX=9606 GN=LCE6A PE=1 SV=1',
 'MSQQKQQSWKPPNVPKCSPPQRSNPCLAPYSTPCGAPHSEGCHSSSQRPEVQKPRRARQKLRCLSRGTTYHCKEEECEGD',
 '>tr|A0A2J8VFF3|A0A2J8VFF3_PONAB LCE6A isoform 1 OS=Pongo abelii OX=9601 GN=LCE6A PE=4 SV=1',
 'MSQQKQQSWKPPNVPKCSPPQRSNPCLAPYSTPCGAPHSEGCHSSSQRPEVQKPRRARQKLRCLSGGTTYHCKEEECEGD',
 '>tr|A0A2R8ZDI4|A0A2R8ZDI4_PANPA Late cornified envelope 6A OS=Pan paniscus OX=9597 GN=LCE6A PE=4 SV=1',
 'MSQQKQQSWKPPNVPKCSPPQRSNPCLTPYSTPCGAPHSEGCHSSSQRPEVQKPRRARQKLRCLSGGTTYHCKEEECEGD',
 '>tr|G3RZ01|G3RZ01_GORGO Late cornified envelope 6A OS=Gorilla gorilla gorilla OX=9595 GN=LCE6A PE=4 SV=1',
 'MSQQKQQSWKPPNVPKCSPPQRSNPCLAPYSTPCGAPHSEGCHSSSQRPEVQKPRRARQKLRCLSGGTIYHCKEEECEGD',
 '>tr|A0A6D2Y655|A0A6D2Y655_PANTR LCE6A isoform 1 OS=Pa